In [62]:
import sys  
sys.path.insert(0, './../')
import numpy as np
import bbdc2021
import pandas as pd
from importlib import reload
reload(bbdc2021)
import tensorflow as tf
from keras import layers
import keras
from keras import backend as K
from itertools import groupby

In [63]:
X_train_orig, Y_train_orig, labelList, timepoints = bbdc2021.read_train("./../data/dataset_fft")
X_train_cropped = [np.delete(x,1,0) for x in X_train_orig]
Y_train_cropped = [np.delete(x,1,0) for x in Y_train_orig]
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = bbdc2021.splitTrain(X_train_cropped, Y_train_cropped)

In [3]:
channels = [32, 64, 128]
inputShape = X_train[0].shape
print(inputShape)

#input_layer = layers.Input(shape=(296,296,1))
input_layer = layers.Input(shape=(inputShape))
##############################
x = input_layer
# Encoder Start
x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
sh1 = layers.ReLU()(x)

x = layers.MaxPooling1D()(sh1)
#############################
x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
sh2 = layers.ReLU()(x)
x = layers.MaxPooling1D()(sh2)
#############################
x = layers.Conv1D(channels[2], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[2], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
#Decoder start
x = layers.UpSampling1D(2)(x)

############################
#sh2 = layers.Cropping1D(cropping=10)(sh2)
x = layers.Concatenate()([x,sh2])
x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.UpSampling1D(2)(x)
#############################
#sh1 = layers.Cropping1D(cropping=16)(sh1)
x = layers.Concatenate()([x,sh1])
x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(13, kernel_size=1, activation="softmax", padding='same')(x)

model = keras.models.Model(inputs=input_layer, outputs=x)
model.summary()

(208, 16)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 208, 16)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 208, 32)      1568        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 208, 32)      128         conv1d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 208, 32)      0           batch_normalization_1[0][0]      
__________________________________________________________________________________

In [ ]:
#Mit crossentropy
model = keras.models.Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['mae'])
history = model.fit(np.array(X_train),np.array(Y_train), batch_size=15, epochs=15, validation_data=(np.array(X_validation), np.array(Y_validation)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/15
8000/8000 [==============================] - 72s 9ms/step - loss: 0.2257 - mae: 0.0136 - val_loss: 0.9747 - val_mae: 0.0494
Epoch 2/15
8000/8000 [==============================] - 59s 7ms/step - loss: 0.2077 - mae: 0.0137 - val_loss: 1.5624 - val_mae: 0.0667
Epoch 3/15
7815/8000 [============================>.] - ETA: 2s - loss: 0.2011 - mae: 0.0135

In [9]:
#Mit Dice
model = keras.models.Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss=bbdc2021.dice_loss, metrics=['mae'])
history = model.fit(np.array(X_train),np.array(Y_train), batch_size=15, epochs=15, validation_data=(np.array(X_validation), np.array(Y_validation)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/15
8000/8000 [==============================] - 61s 8ms/step - loss: 0.1780 - mae: 0.0629 - val_loss: 0.2708 - val_mae: 0.0806
Epoch 2/15
8000/8000 [==============================] - 52s 6ms/step - loss: 0.1234 - mae: 0.0425 - val_loss: 0.3198 - val_mae: 0.0970
Epoch 3/15
8000/8000 [==============================] - 52s 7ms/step - loss: 0.1043 - mae: 0.0356 - val_loss: 0.3833 - val_mae: 0.1051
Epoch 4/15
8000/8000 [==============================] - 55s 7ms/step - loss: 0.0928 - mae: 0.0315 - val_loss: 0.2020 - val_mae: 0.0628
Epoch 5/15
8000/8000 [==============================] - 55s 7ms/step - loss: 0.0847 - mae: 0.0287 - val_loss: 0.1799 - val_mae: 0.0583
Epoch 6/15
8000/8000 [==============================] - 55s 7ms/step - loss: 0.0790 - mae: 0.0268 - val_loss: 0.1970 - val_mae: 0.0585
Epoch 7/15
8000/8000 [==============================] - 58s 7ms/step - loss: 0.0740 - mae: 0.0250 - val_loss: 0.2961 - val_mae: 0.0865
Epoch 8

In [ ]:
#Mit binary_crossentropy
model = keras.models.Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mae'])
history = model.fit(np.array(X_train),np.array(Y_train), batch_size=15, epochs=15, validation_data=(np.array(X_validation), np.array(Y_validation)))

In [132]:
from operator import itemgetter
y_predicted = np.argmax(model.predict(np.array(X_test)), axis=2)
y_predictedNew = np.zeros((y_predicted.shape[0]+1, y_predicted.shape[1]))
y_predictedNew[0] = timepoints[1:]
y_predictedNew[1:] = y_predicted

result = np.transpose(y_predictedNew[0:2])
result 
a = [list(group) for key, group in groupby(result, itemgetter(1))]
a[0]

[array([ 0.048, 12.   ]),
 array([ 0.096, 12.   ]),
 array([ 0.144, 12.   ]),
 array([ 0.192, 12.   ]),
 array([ 0.24, 12.  ]),
 array([ 0.288, 12.   ]),
 array([ 0.336, 12.   ]),
 array([ 0.384, 12.   ]),
 array([ 0.432, 12.   ]),
 array([ 0.48, 12.  ]),
 array([ 0.528, 12.   ]),
 array([ 0.576, 12.   ]),
 array([ 0.624, 12.   ]),
 array([ 0.672, 12.   ]),
 array([ 0.72, 12.  ]),
 array([ 0.768, 12.   ]),
 array([ 0.816, 12.   ]),
 array([ 0.864, 12.   ]),
 array([ 0.912, 12.   ]),
 array([ 0.96, 12.  ]),
 array([ 1.008, 12.   ]),
 array([ 1.056, 12.   ]),
 array([ 1.104, 12.   ]),
 array([ 1.152, 12.   ]),
 array([ 1.2, 12. ])]

In [86]:
result = labelList[y_predicted][0]

array([[ 0.048,  0.096,  0.144, ...,  9.888,  9.936,  9.984],
       [12.   , 12.   , 12.   , ..., 12.   , 12.   , 12.   ],
       [12.   , 12.   , 12.   , ..., 12.   , 12.   , 12.   ],
       ...,
       [12.   , 12.   , 12.   , ..., 12.   , 12.   , 12.   ],
       [12.   , 12.   ,  0.   , ..., 12.   , 12.   , 12.   ],
       [12.   , 12.   , 12.   , ..., 12.   , 12.   , 12.   ]])